In [ ]:
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tqdm.auto import tqdm
import torch

def ir_srgan(img_path, save_path, netG):
    # Image saving path
    img_name = img_path[img_path.rfind("\\")+1:]
    print("Image name: ",img_name)
    lr_img = Image.open(img_path)
    if lr_img.size[0] == 3:
        lr_img = lr_img.convert('L')
    pre_process_lr = transforms.Compose([transforms.ToTensor()])

    lr_img = pre_process_lr(lr_img)
    print("Shape of the input image: ",lr_img.shape)
    # show the image
    lr_img = lr_img.permute(1, 2, 0)
    print("Input image: ")
    plt.imshow(lr_img)
    plt.show()

    import torch
    with torch.no_grad():
        lr_img = lr_img.permute(2, 0, 1)
        sr_img = netG(lr_img.unsqueeze(0)) # c, h, w
        print("Shape of the Super Resolved image: ",sr_img.shape)
        sr_img = sr_img.squeeze(0).permute(1, 2, 0).clamp(0, 1).numpy()
        ing_file_name = os.path.join(save_path,img_name)
        sr_img = np.squeeze(sr_img)
        plt.imsave(ing_file_name, sr_img, cmap='gray')
        plt.imshow(sr_img)
        plt.show()

def ir_srgan_bicubic(img_path, save_path, netG):
    # Image saving path
    img_name = img_path[img_path.rfind("\\")+1:]
    print("Image name: ",img_name)
    lr_img = Image.open(img_path)
    _ , H, W = lr_img.size
    if lr_img.size[0] == 3:
        lr_img = lr_img.convert('L')
    pre_process_lr = transforms.Compose([transforms.Resize((H//4, W//4), interpolation=Image.BICUBIC),
                                        transforms.ToTensor()])

    lr_img = pre_process_lr(lr_img)
    print("Shape of the input image: ",lr_img.shape)
    # show the image
    lr_img = lr_img.permute(1, 2, 0)
    print("Input image: ")
    plt.imshow(lr_img)
    plt.show()

 
    with torch.no_grad():
        lr_img = lr_img.permute(2, 0, 1)
        sr_img = netG(lr_img.unsqueeze(0)) # c, h, w
        print("Shape of the Super Resolved image: ",sr_img.shape)
        sr_img = sr_img.squeeze(0).permute(1, 2, 0).clamp(0, 1).numpy()
        ing_file_name = os.path.join(save_path,img_name)
        sr_img = np.squeeze(sr_img)
        plt.imsave(ing_file_name, sr_img, cmap='gray')
        plt.imshow(sr_img)
        plt.show()

### Model Evaluation for IR Super resolution GAN 
* Pixel loss - L1
* Feature loss - L1
* No edge loss

In [ ]:
%cd C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs
import torch
import torch.nn as nn
from RRDBNet_arch import RRDBNet

ir_edge_netG = RRDBNet(in_nc=1, out_nc=1, nf=64, nb=23, gc=32)

#load the otsr model
# model parameters when the psnr is around 21
path_0 = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_L1\9494_G.pth'

# model parameters at the end of training at 40000th step
path_1 = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_L1\40138_G.pth'
ir_edge_netG.load_state_dict(torch.load(path_1), strict=True)

## Direct upscaling images

Upscaling the original images without downscaling. 

#### Validation for FLIR Images 

dataset path = C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\FLIR_Dataset

In [ ]:
gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\FLIR_Dataset"
img_list = os.listdir(gt_img)
img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.jpeg')]
save_path = os.path.join(gt_img,"direct_upsampled_images_PL1_FL1_1")
os.makedirs(save_path, exist_ok=True)
for ixd in tqdm(range(len(img_path))):
    img_ = img_path[ixd]
    ir_srgan(img_, save_path, ir_edge_netG)

#### Validation on DroneImages 
Dataset path = C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\DroneImages

In [ ]:
gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\DroneImages"
img_list = os.listdir(gt_img)
img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.jpg')]
save_path = os.path.join(gt_img,"direct_upsampled_images_PL1_FL1_1")
os.makedirs(save_path, exist_ok=True)
for ixd in tqdm(range(len(img_path))):
    img_ = img_path[ixd]
    ir_srgan(img_, save_path, ir_edge_netG)

#### Validation of pedistrainImages
Dataset path = C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\pedistrainImages

In [ ]:
gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\pedistrainImages"
img_list = os.listdir(gt_img)
img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.bmp')]
save_path = os.path.join(gt_img,"direct_upsampled_images_PL1_FL1_0")
os.makedirs(save_path, exist_ok=True)
for ixd in tqdm(range(len(img_path))):
    img_ = img_path[ixd]
    ir_srgan(img_, save_path, ir_edge_netG)

### Model Evaluation for IR Super resolution GAN 
* Pixel loss - L2
* Feature loss - L2
* No edge loss

In [ ]:
%cd C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs
import torch
import torch.nn as nn
from RRDBNet_arch import RRDBNet

ir_edge_netG = RRDBNet(in_nc=1, out_nc=1, nf=64, nb=23, gc=32)

#load the otsr model
# model parameters when the psnr is around 24
path_0 = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_PL2_FL2\40138_G.pth'

# model parameters at the end of training at 40000th step
path_1 = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_PL2_FL2\40138_G.pth'
ir_edge_netG.load_state_dict(torch.load(path_0), strict=True)

In [ ]:
gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\FLIR_Dataset"
img_list = os.listdir(gt_img)
img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.jpeg')]
save_path = os.path.join(gt_img,"direct_upsampled_images_PL2_FL2")
os.makedirs(save_path, exist_ok=True)
for ixd in tqdm(range(len(img_path))):
    img_ = img_path[ixd]
    ir_srgan(img_, save_path, ir_edge_netG)

In [ ]:
gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\DroneImages"
img_list = os.listdir(gt_img)
img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.jpg')]
save_path = os.path.join(gt_img,"direct_upsampled_images_PL2_FL2")
os.makedirs(save_path, exist_ok=True)
for ixd in tqdm(range(len(img_path))):
    img_ = img_path[ixd]
    ir_srgan(img_, save_path, ir_edge_netG)

### Model Evaluation for IR Super resolution GAN 
* Pixel loss - L1
* Feature loss - L2
* No edge loss

In [ ]:
%cd C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs
import torch
import torch.nn as nn
from RRDBNet_arch import RRDBNet

ir_edge_netG = RRDBNet(in_nc=1, out_nc=1, nf=64, nb=23, gc=32)

#load the otsr model
# model parameters when the psnr is around 24
path_0 = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_PL1_FL2\40138_G.pth'

# model parameters at the end of training at 40000th step
path_1 = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_PL2_FL2\40138_G.pth'
ir_edge_netG.load_state_dict(torch.load(path_0), strict=True)

In [ ]:
gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\FLIR_Dataset"
img_list = os.listdir(gt_img)
img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.jpeg')]
save_path = os.path.join(gt_img,"direct_upsampled_images_PL1_FL2")
os.makedirs(save_path, exist_ok=True)
for ixd in tqdm(range(len(img_path))):
    img_ = img_path[ixd]
    ir_srgan(img_, save_path, ir_edge_netG)


gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\DroneImages"
img_list = os.listdir(gt_img)
img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.jpg')]
save_path = os.path.join(gt_img,"direct_upsampled_images_PL1_FL2")
os.makedirs(save_path, exist_ok=True)
for ixd in tqdm(range(len(img_path))):
    img_ = img_path[ixd]
    ir_srgan(img_, save_path, ir_edge_netG)

### Model Evaluation for IR Super resolution GAN 
* Pixel loss - L1
* Feature loss - L2
* Edge loss - L1 1e

In [ ]:
%cd C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs
import torch
import torch.nn as nn
from RRDBNet_arch import RRDBNet

ir_edge_netG = RRDBNet(in_nc=1, out_nc=1, nf=64, nb=23, gc=32)

#load the otsr model
# model parameters when the psnr is around 24
path_0 = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_with_edgeloss\trained_models_sobel\40138_G.pth'

# model parameters at the end of training at 40000th step
path_1 = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_PL2_FL2\40138_G.pth'
ir_edge_netG.load_state_dict(torch.load(path_0), strict=True)

In [ ]:
gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\FLIR_Dataset"
img_list = os.listdir(gt_img)
img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.jpeg')]
save_path = os.path.join(gt_img,"direct_upsampled_images_sobel_PL1_FL2")
os.makedirs(save_path, exist_ok=True)
for ixd in tqdm(range(len(img_path))):
    img_ = img_path[ixd]
    ir_srgan(img_, save_path, ir_edge_netG)


gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\DroneImages"
img_list = os.listdir(gt_img)
img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.jpg')]
save_path = os.path.join(gt_img,"direct_upsampled_images_sobel_PL1_FL2")
os.makedirs(save_path, exist_ok=True)
for ixd in tqdm(range(len(img_path))):
    img_ = img_path[ixd]
    ir_srgan(img_, save_path, ir_edge_netG)

# Bicubic Downscaling and upscaling


In [ ]:
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tqdm.auto import tqdm
import torch

def ir_srgan_bicubic(img_path, save_path, netG):
    # Image saving path
    img_name = img_path[img_path.rfind("\\")+1:]
    print("Image name: ",img_name)
    
    lr_img = Image.open(img_path)
    print("Shape of the input image: ",lr_img.size)
    if lr_img.size[0]!=3:
        H, W = lr_img.size
    if lr_img.size[0] == 3:
        lr_img = lr_img.convert('L')
    pre_process_lr = transforms.Compose([transforms.Resize((W//4, H//4), interpolation=Image.BICUBIC),
                                        transforms.ToTensor()])

    lr_img = pre_process_lr(lr_img)
    print("Shape of the input image: ",lr_img.shape)
    # show the image
    lr_img = lr_img.permute(1, 2, 0)
    print("Input image: ")
    plt.imshow(lr_img)
    plt.show()

 
    with torch.no_grad():
        lr_img = lr_img.permute(2, 0, 1)
        sr_img = netG(lr_img.unsqueeze(0)) # c, h, w
        print("Shape of the Super Resolved image: ",sr_img.shape)
        sr_img = sr_img.squeeze(0).permute(1, 2, 0).clamp(0, 1).numpy()
        ing_file_name = os.path.join(save_path,img_name)
        sr_img = np.squeeze(sr_img)
        plt.imsave(ing_file_name, sr_img, cmap='gray')
        plt.imshow(sr_img)
        plt.show()

In [ ]:
# %cd C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs

def modelValidation(model_paths):
    for model_name, model_path in model_paths.items():
        %cd C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs

        import torch
        import torch.nn as nn
        from RRDBNet_arch import RRDBNet

        ir_edge_netG = RRDBNet(in_nc=1, out_nc=1, nf=64, nb=23, gc=32)

        ir_edge_netG.load_state_dict(torch.load(model_path), strict=True)

        ### in Leo
        # gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\FLIR_Dataset"

        ### in Sai
        gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\FLIR_Dataset"
        img_list = os.listdir(gt_img)
        img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.jpeg')]
        save_path = os.path.join(gt_img,f"EVAL_02_test_{model_name}")
        os.makedirs(save_path, exist_ok=True)
        for ixd in tqdm(range(len(img_path))):
            img_ = img_path[ixd]
            ir_srgan_bicubic(img_, save_path, ir_edge_netG)


        gt_img = r"C:\SaiVinay\GitHub\ir_srgan_otsr\model_validation\validation_datasets\DroneImages"
        img_list = os.listdir(gt_img)
        img_path = [os.path.join(gt_img, x) for x in img_list if x.endswith('.jpg')]
        save_path = os.path.join(gt_img,f"EVAL_02_test_{model_name}")
        os.makedirs(save_path, exist_ok=True)
        for ixd in tqdm(range(len(img_path))):
            img_ = img_path[ixd]
            ir_srgan_bicubic(img_, save_path, ir_edge_netG)

In [ ]:
model_paths = {
    "PL1_FL1": r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_L1\9494_G.pth',
    "PL2_FL2": r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_PL2_FL2\40138_G.pth',
    "PL1_FL2": r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_PL1_FL2\40138_G.pth',
    "PL1_5e_FL2": r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_PW5e\40138_G.pth',
    "sobel": r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_with_edgeloss\trained_models_sobel\40138_G.pth',
    "sobel_5e": r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_with_edgeloss\trained_models_edgeloss5e\40138_G.pth'
}

modelValidation(model_paths)

### Calculate the Average PSNR 